# BERT Fine-Tuning Sentence Classification

> BERT Fine-Tuning Tutorial with PyTorch
> 
> ref: https://colab.research.google.com/drive/1Y4o3jh3ZH70tl6mCd76vz_IxX23biCPP

- toc: true 
- badges: true
- comments: true
- categories: [bert, jupyter]

# 1. Setup
## 1.1. Using Colab GPU for Training

In [1]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [2]:
import torch

# If there's a GPU available,
if torch.cuda.is_available():

    # Tell PyTorch to user the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will user the GPU:', torch.cuda.get_device_name(0))

# If not,
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


There are 1 GPU(s) available.
We will user the GPU: Tesla T4


## 1.2. Installing the Hugging Face Library

In [0]:
! pip install transformers -q

In [0]:
! pip install wget -q

In [5]:
import wget 
import os

print('Downloading dataset...')

# The URL for the dataset zip file.
url = 'https://nyu-mll.github.io/CoLA/cola_public_1.1.zip'

# Download the file (if we haven't already)
if not os.path.exists('./cola_public_1.1.zip'):
    wget.download(url, './cola_public_1.1.zip')

In [0]:
# Unzip the dataset (if we haven't already)
if not os.path.exists('./cola_public/'):
    ! unzip cola_public_1.1.zip

In [7]:
! head cola_public/*/*

==> cola_public/raw/in_domain_dev.tsv <==
gj04	1		The sailors rode the breeze clear of the rocks.
gj04	1		The weights made the rope stretch over the pulley.
gj04	1		The mechanical doll wriggled itself loose.
cj99	1		If you had eaten more, you would want less.
cj99	0	*	As you eat the most, you want the least.
cj99	0	*	The more you would want, the less you would eat.
cj99	0	*	I demand that the more John eat, the more he pays.
cj99	1		Mary listens to the Grateful Dead, she gets depressed.
cj99	1		The angrier Mary got, the more she looked at pictures.
cj99	1		The higher the stakes, the lower his expectations are.

==> cola_public/raw/in_domain_train.tsv <==
gj04	1		Our friends won't buy this analysis, let alone the next one we propose.
gj04	1		One more pseudo generalization and I'm giving up.
gj04	1		One more pseudo generalization or I'm giving up.
gj04	1		The more we study verbs, the crazier they get.
gj04	1		Day by day the facts are getting murkier.
gj04	1		I'll fix you a drink.
gj04	1		

## 2.2. Parse

In [8]:
import pandas as pd

# Load the dataset into a pandas dataframe.
df = pd.read_csv("./cola_public/raw/in_domain_train.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data
df.sample(10)

Number of training sentences: 8,551



,sentence_source,label,label_notes,sentence
1643,r-67,1,NaN,I brought a razor with which to shave myself.
558,bc01,1,NaN,A bear inhabits the cave.
7840,ad03,1,NaN,The therapist's analysis of Lucy
4713,ks08,1,NaN,This noise cannot be put up with.
5501,b_73,1,NaN,Such a scholar as you were speaking of just no...
568,bc01,1,NaN,The rolling stone avoided the river.
1920,r-67,1,NaN,I would prefer it for there to be no talking.
2492,l-93,1,NaN,He worked his way through the book.
5364,b_73,1,NaN,Enough is going on to keep them confused.
5344,b_73,1,NaN,Mary is taller than six feet.


In [9]:
df[df.label == 0].sample(5)[['sentence', 'label']]

,sentence,label
444,Mickey slips all the time up.,0
187,"The more books I ask to whom he will give, the...",0
4866,This needs investigating the problem.,0
7780,We believed to be omnipotent.,0
1267,What table will he put the chair between and s...,0


In [0]:
# Get the lists of sentences and their labels.
sentences = df.sentence.values
labels = df.label.values

# 3. Tokenization & Input Formatting

## 3.1. BERT Tokenizer

In [11]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [12]:
# Print the original sentence.
print(' Original: ', sentences[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

 Original:  Our friends won't buy this analysis, let alone the next one we propose.
Tokenized:  ['our', 'friends', 'won', "'", 't', 'buy', 'this', 'analysis', ',', 'let', 'alone', 'the', 'next', 'one', 'we', 'propose', '.']
Token IDs:  [14008, 16119, 11441, 112, 162, 35172, 10372, 15559, 117, 12421, 19145, 10103, 12878, 10399, 11312, 25690, 119]


In [13]:
korean = "안녕하세요. 반갑습니다. 너는 이름이 뭐니? 오늘 날씨가 맑고 좋구나."
# Print the original sentence.
print(' Original: ', korean)

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(korean))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(korean)))

 Original:  안녕하세요. 반갑습니다. 너는 이름이 뭐니? 오늘 날씨가 맑고 좋구나.
Tokenized:  ['ᄋ', '##ᅡᆫ', '##녀', '##ᆼ', '##하', '##세', '##요', '.', 'ᄇ', '##ᅡᆫ', '##가', '##ᆸ', '##스', '##ᆸ니다', '.', 'ᄂ', '##ᅥ', '##는', '이', '##름이', 'ᄆ', '##ᅯ', '##니', '?', 'ᄋ', '##ᅩ', '##ᄂ', '##ᅳᆯ', '날', '##씨', '##가', 'ᄆ', '##ᅡ', '##ᆰ', '##고', 'ᄌ', '##ᅩ', '##ᇂ', '##구', '##나', '.']
Token IDs:  [1174, 26646, 49345, 13045, 35132, 25169, 47024, 119, 1170, 26646, 11376, 17360, 13212, 79427, 119, 1165, 33645, 11192, 12398, 89420, 1169, 97090, 25536, 136, 1174, 29347, 97071, 63277, 76818, 47928, 11376, 1169, 25539, 97098, 12300, 1175, 29347, 97109, 16336, 16801, 119]


When we actually convert all of our sentences, we'll use the `tokenize.encode` functio to handle both steps, rather than calling `tokenize` and `convert_tokens_to_ids` seperately.

## 3.2. Required Formatting

## 3.3. Tokenize Dataset

In [14]:
max_len = 0

# For every sentence,
for sent in sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence langth.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  48


In [15]:
# Tokenize all of the sentences and map the tokens to their word IDs.
input_ids = []
attention_masks = []

# For every sentence,
for sent in sentences:
    # `encode_plus` will:
    # (1) Tokenize the sentence.
    # (2) Prepend the `[CLS]` token to the start.
    # (3) Append the `[SEP]` token to the end.
    # (4) Map tokens to their IDs.
    # (5) Pad or truncate the sentence to `max_length`
    # (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
        sent,                                       # Sentence to encode.
        add_special_tokens = True,      # Add '[CLS]' and '[SEP]'
        max_length = 64,                           # Pad & truncate all sentences.
        pad_to_max_length = True,
        return_attention_mask = True,   # Construct attn. masks.
        return_Tensors = 'pt',                  # Return pytorch tensors.
    )
    
    # Add the encoded sentence to the list.
    input_ids.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding.)
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.LongTensor(input_ids)
attention_masks = torch.FloatTensor(attention_masks)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original:  ', sentences[0])
print('Token IDs: ', input_ids[0])

Original:   Our friends won't buy this analysis, let alone the next one we propose.
Token IDs:  tensor([  101, 14008, 16119, 11441,   112,   162, 35172, 10372, 15559,   117,
        12421, 19145, 10103, 12878, 10399, 11312, 25690,   119,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])


## 3.4. Training & Validation Split

In [16]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Devide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5} training samples'.format(train_size))
print('{:>5} validation samples'.format(val_size))

 7695 training samples
  856 validation samples


In [0]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it here.
# For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order.
train_dataloader = DataLoader(
    train_dataset,   # The training samples.
    sampler = RandomSampler(train_dataset),   # Select batches randomly
    batch_size = batch_size   # Trains with this batch size.
)

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
    val_dataset,   # The validation samples.
    sampler = SequentialSampler(val_dataset),   # Pull out batches sequentially.
    batch_size = batch_size   # Evaluate with this batch size.
)

In [18]:
train_dataloader.batch_size
# 7695/32 = 240.46875

32

# 4. Train Our Classification Model

## 4.1. BertForSequenceClassification

* BertModel
* BertForPreTraining
* BertForMaskedLM
* BertForNextSentencePredicion
* BertForSequenceClassification
* BertForTokenClassification
* BertForQuestionAnswering

In [19]:
device

device(type='cuda')

In [20]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with 
# a single linear classification layer on top.
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2,    # The number of output labels--2 for binary classification.
                                # You can increase this for multi-class tasks.
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
# if device == 'cuda':
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_aff

In [21]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

## 4.2. Optimizer & Learning Rate Scheduler

In [0]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch)
# I believe the 'W' stands for 'Weight Decay fix'

optimizer = AdamW(model.parameters(),
                  lr = 2e-5,   # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.dam_epsilon  - default is 1e-8
                  )

In [0]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4.
# We chose to run for 4, but we'll see later that this my be over-fitting the training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

## 4.3. Training Loop

**Training:**
* Unpack our data inputs and labels
* Load data onto the GPU for acceleration
* Clear out the gradients calculated in the previous pass.
    - In pytorch the gradients accumulate by default (useful for things like RNNs) unless you explicitly clear them out.
* Forward pass (feed input data through the network)
* Backward pass (backpropagation)
* Tell the network to update parameters with optimizer.step()
* Track variables for monitoring progress

**Evaluation:**
* Unpack our data inputs and labels
* Load data onto the GPU for acceleration
* Forward pass (feed input data through the network)
* Compute loss on our validation data and track variables for monitoring progress

In [0]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    print("    ", elapsed, elapsed_rounded, str(datetime.timedelta(seconds=elapsed_rounded)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [26]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss,
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch,
for epoch_i in range(0, epochs):

    # ========================================
    #                                 Training 
    # ========================================

    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i+1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training vs test
    # (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data,
    for step, batch in enumerate(train_dataloader):
        # # of step : 241
        # len of batch : 32
        # size of batch[0] : 64
        
        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            
            # Calcuate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)

            # Report progress.
            print('  Batch {:>5,} of {:>5,}.    Elapsed: {:}'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using `to` method.
        #
        # `batch` contains three pytorch tensors:
        #     [0]: input ids
        #     [1] : attention masks
        #     [2] : labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        print(type(batch[0]))
        print(batch[0])
        print(type(b_input_ids))
        print(b_input_ids)

        # Always clear any previously calculated gradients before performing backward pass.
        # PyTorch doesn't do this automatically because accumulating the gradients is 
        # "convenient while training RNNs". (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()

        # Perform a forwad pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here:
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # are given and what flags are set. For our usage here, it returns the loss (because we provided labels)
        # and the "logits"--the model outputs prior to activation.
        loss, logits = model(b_input_ids,
                                        token_type_ids=None,
                                        attention_mask=b_input_mask,
                                        labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can calculate the average loss at the end.
        # `loss` is a Tensor containing a single value; the `.item()` function just returns the Python value from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are modified 
        # based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))

    # ========================================
    #                                 Validation 
    # ========================================
    # After the completion of each training epoch, measure our performance on our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently during evaluation.
    model.eval()

    # Tracking variables
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Tell pytorch not to bother with constructing the compute graph during the forward pass,
        # since this is only needed for backprop (training).
        with torch.no_grad():

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here:
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output byt the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids,
                                                token_type_ids=None,
                                                attention_mask=b_input_mask,
                                                labels=b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentence,
        # and accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)

    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

    print("")
    print("Training complete!")

    print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...
<class 'torch.Tensor'>
tensor([[  101, 11312, 10320,  ...,     0,     0,     0],
        [  101, 13667, 10564,  ...,     0,     0,     0],
        [  101, 10103, 23222,  ...,     0,     0,     0],
        ...,
        [  101, 21506, 10935,  ...,     0,     0,     0],
        [  101, 69042, 10127,  ...,     0,     0,     0],
        [  101, 12941, 50134,  ...,     0,     0,     0]])
<class 'torch.Tensor'>
tensor([[  101, 11312, 10320,  ...,     0,     0,     0],
        [  101, 13667, 10564,  ...,     0,     0,     0],
        [  101, 10103, 23222,  ...,     0,     0,     0],
        ...,
        [  101, 21506, 10935,  ...,     0,     0,     0],
        [  101, 69042, 10127,  ...,     0,     0,     0],
        [  101, 12941, 50134,  ...,     0,     0,     0]], device='cuda:0')


RuntimeError: ignored